In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from warnings import filterwarnings


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc

filterwarnings('ignore')

nltk.download('stopwords')
nltk.download('punkt')
plt.style.use('dark_background')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Denis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Denis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Загружаем стоп-слова из библиотеки NLTK
stop_words = set(stopwords.words('english'))

In [5]:
# Считываем датасет
df = pd.read_csv('FakeNewsNet.csv', encoding='utf-8', usecols=['title', 'news_url', 'source_domain', 'real'])
df = df.dropna(how='any', axis=0)
print(f'Cтрок после удаления: {df.shape[0]}')

df.head()

Cтрок после удаления: 22866


,title,news_url,source_domain,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,1


In [11]:
df['text'] = df.apply(lambda x: x['title']+'; '+x['news_url']+'; '+x['source_domain'], axis=1)
df_0 = df[['text', 'real']].copy()
df_0.head(5)

,text,real
0,Kandi Burruss Explodes Over Rape Accusation on...,1
1,People's Choice Awards 2018: The best red carp...,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,1
3,Colombian singer Maluma sparks rumours of inap...,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,1


In [6]:
# Проверяем на наличие null-значений`a
df.isnull().sum()

title            0
news_url         0
source_domain    0
real             0
dtype: int64

In [ ]:
# Количество элементов каждого класса
class_counts = df['sentiment'].value_counts()

# Визуализация графика баланса классов
plt.figure(figsize=(6, 4))
sns.countplot(x='sentiment', data=df, hue='sentiment', palette='Set2') #, legend=False)
plt.title('Баланс классов')
plt.xlabel('Классы')
plt.ylabel('Количество элементов')
plt.show()

In [ ]:
def preprocess_text(text):
    # Удаляем знаки препинания
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Приводим к нижнему регистру
    text = text.lower()

    # Токенизация текста
    tokens = word_tokenize(text)

    # Удаляем стоп-слова
    tokens = [word for word in tokens if word.lower() not in stop_words]

    # Удаляем числа и другие символы
    tokens = [word for word in tokens if word.isalpha()]

    # Склеиваем токены обратно в строку
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [ ]:
def preprocess_dataframe(dataframe, text_column):
    # Применяем preprocess_text ко всем значениям в указанной колонке
    dataframe[text_column] = dataframe[text_column].apply(preprocess_text)

    return dataframe

In [ ]:
preprocess_dataframe(df, 'review')

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['review'].tolist())

In [ ]:
# Получение словаря
word_index = tokenizer.word_index

In [ ]:
sequences = tokenizer.texts_to_sequences(df['review'].tolist())
data = sequence.pad_sequences(sequences)
print(data)

In [ ]:
# Кодируем категориальный признак
category_mapping = {'positive': 1, 'negative': 0}

In [ ]:
df['label'] = df['sentiment'].replace(category_mapping)

In [ ]:
# Разделение на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(data, df['label'], test_size=0.20, random_state=42)

# LSTM

In [ ]:
vocab_size = len(tokenizer.word_index) + 1  # добавляем 1, так как индексы начинаются с 1
max_length = max(len(seq) for seq in sequences)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length),
    tf.keras.layers.LSTM(units=64),
    tf.keras.layers.Dense(1, activation='sigmoid')  # для бинарной классификации
])

# Используем early_stop, который прекращает обучение, когда validation loss больше не улучшается
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=30, shuffle=True, callbacks=[early_stop])

In [ ]:
history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']
epochs = history.epoch

plt.figure(figsize=(6,4))
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss', size=10)
plt.xlabel('Epochs', size=10)
plt.ylabel('Loss', size=10)
plt.legend(prop={'size': 10})
plt.show()

plt.figure(figsize=(6,4))
plt.plot(epochs, acc, 'g', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy', size=10)
plt.xlabel('Epochs', size=10)
plt.ylabel('Accuracy', size=10)
plt.legend(prop={'size': 10})
plt.ylim((0.5,1))
plt.show()

In [ ]:
# Получаем предсказания
y_pred = model.predict(X_test)

# Округляем вероятности до 0 или 1
y_pred_binary = np.round(y_pred)

# Оцениваем производительность модели
accuracy = accuracy_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
roc_auc = roc_auc_score(y_test, y_pred)

# Выводим результаты
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)

# Строим ROC-кривую
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()